In [ ]:
from huggingface_hub import hf_hub_download

# Arc2Face 모델
hf_hub_download("FoivosPar/Arc2Face", "arc2face/config.json",           local_dir="models/arc2face")
hf_hub_download("FoivosPar/Arc2Face", "arc2face/diffusion_pytorch_model.safetensors", local_dir="models/arc2face")
hf_hub_download("FoivosPar/Arc2Face", "encoder/config.json",             local_dir="models/encoder")
hf_hub_download("FoivosPar/Arc2Face", "encoder/pytorch_model.bin",       local_dir="models/encoder")

# AntelopeV2 + ArcFace ONNX
!mkdir -p models/antelopev2
# (zip 해제 or hf_hub_download로 arcface.onnx까지 models/antelopev2에 위치)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Colab에서 실행
!mkdir -p models/antelopev2
!unzip "/content/drive/MyDrive/antelopev2.zip" -d models/antelopev2

Archive:  /content/drive/MyDrive/antelopev2.zip
   creating: models/antelopev2/antelopev2/
  inflating: models/antelopev2/antelopev2/genderage.onnx  
  inflating: models/antelopev2/antelopev2/2d106det.onnx  
  inflating: models/antelopev2/antelopev2/1k3d68.onnx  
  inflating: models/antelopev2/antelopev2/glintr100.onnx  
  inflating: models/antelopev2/antelopev2/scrfd_10g_bnkps.onnx  


In [ ]:
hf_hub_download(repo_id="FoivosPar/Arc2Face", filename="arcface.onnx", local_dir="./models/antelopev2")

'./models/antelopev2/arcface.onnx'

In [ ]:
%pip install --upgrade \
  "huggingface_hub==0.17.3" \
  "transformers==4.41.0" \
  "diffusers==0.23.0" \
  "accelerate==0.20.3"


  Using cached huggingface_hub-0.17.3-py3-none-any.whl.metadata (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install huggingface_hub==0.17.3 and transformers==4.41.0 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested huggingface_hub==0.17.3
    transformers 4.41.0 depends on huggingface-hub<1.0 and >=0.23.0

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [ ]:
from diffusers import (
    StableDiffusionPipeline,
    UNet2DConditionModel,
    DPMSolverMultistepScheduler,
)
import sys, os
# Arc2Face 클론 경로에 맞게 변경
sys.path.append(os.path.abspath("/content/Arc2Face"))


from arc2face import CLIPTextModelWrapper, project_face_embs

import torch
from insightface.app import FaceAnalysis
from PIL import Image
import numpy as np

# Arc2Face is built upon SD1.5
# The repo below can be used instead of the now deprecated 'runwayml/stable-diffusion-v1-5'
base_model = 'stable-diffusion-v1-5/stable-diffusion-v1-5'

encoder = CLIPTextModelWrapper.from_pretrained(
    'models', subfolder="encoder", torch_dtype=torch.float16
)

unet = UNet2DConditionModel.from_pretrained(
    'models', subfolder="arc2face", torch_dtype=torch.float16
)

pipeline = StableDiffusionPipeline.from_pretrained(
        base_model,
        text_encoder=encoder,
        unet=unet,
        torch_dtype=torch.float16,
        safety_checker=None
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [ ]:
from diffusers import StableDiffusionPipeline, UNet2DConditionModel, DPMSolverMultistepScheduler
from arc2face import CLIPTextModelWrapper
import torch

# encoder, unet 로드
encoder = CLIPTextModelWrapper.from_pretrained('models', subfolder="encoder", torch_dtype=torch.float16)
unet    = UNet2DConditionModel.from_pretrained('models', subfolder="arc2face", torch_dtype=torch.float16)

# 파이프라인 생성
pipeline = StableDiffusionPipeline.from_pretrained(
    'stable-diffusion-v1-5',
    text_encoder=encoder,
    unet=unet,
    torch_dtype=torch.float16,
    safety_checker=None
)

# 스케줄러 교체 및 CUDA 이동
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)
pipeline = pipeline.to('cuda')


Couldn't connect to the Hub: 401 Client Error. (Request ID: Root=1-680d0c3e-19d4a800669605da291e0ee8;a0db71fd-2a13-4d87-b83d-1977abd94548)

Repository Not Found for url: https://huggingface.co/api/models/stable-diffusion-v1-5.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password..
Will try to load from local cache.


OSError: Cannot load model stable-diffusion-v1-5: model is not cached locally and an error occured while trying to fetch metadata from the Hub. Please check out the root cause in the stacktrace above.

In [ ]:
# 1) 맞춤 버전 설치: cached_download 도 있고 split_* 도 있는 버전
%pip install huggingface_hub==0.25.2

# 2) (선택) diffusers, transformers, accelerate 버전 고정
%pip install --upgrade diffusers==0.23.0 transformers==4.34.1 accelerate==0.20.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.30.2
    Uninstalling huggingface-hub-0.30.2:
      Successfully uninstalled huggingface-hub-0.30.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.25.2 which is incompatible.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.34.1 which is incompatible.


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, UNet2DConditionModel, DPMSolverMultistepScheduler
from arc2face import CLIPTextModelWrapper, project_face_embs

# 1) 텍스트 인코더와 U-Net 로드
encoder = CLIPTextModelWrapper.from_pretrained(
    "models", subfolder="encoder", torch_dtype=torch.float16
)
unet = UNet2DConditionModel.from_pretrained(
    "models", subfolder="arc2face", torch_dtype=torch.float16
)

# 2) Stable Diffusion 기반 Arc2Face 파이프라인 생성
pipeline = StableDiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5",
    text_encoder=encoder,
    unet=unet,
    torch_dtype=torch.float16,
    safety_checker=None,
)

# 3) 스케줄러 교체 및 CUDA로 이동
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)
pipeline = pipeline.to("cuda")


Couldn't connect to the Hub: 401 Client Error. (Request ID: Root=1-680d0c1e-0345f40b458ecc4c52230acb;51a124d8-87e6-4f4a-a082-8953e7a57b10)

Repository Not Found for url: https://huggingface.co/api/models/stable-diffusion-v1-5.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password..
Will try to load from local cache.


OSError: Cannot load model stable-diffusion-v1-5: model is not cached locally and an error occured while trying to fetch metadata from the Hub. Please check out the root cause in the stacktrace above.

In [ ]:
from insightface.app import FaceAnalysis
from PIL import Image
import numpy as np

# 1) AntelopeV2 얼굴 분석 객체 준비
app = FaceAnalysis(name="antelopev2", root=".", providers=["CUDAExecutionProvider","CPUExecutionProvider"])
app.prepare(ctx_id=0, det_size=(640,640))

# 2) 예시 이미지 로드 -> 가장 큰 얼굴 선택
img = np.array(Image.open("assets/examples/joacquin.png"))[:,:,::-1]
faces = app.get(img)
# bbox 면적 기준 가장 큰 face
face = sorted(faces, key=lambda x:(x["bbox"][2]-x["bbox"][0])*(x["bbox"][3]-x["bbox"][1]))[-1]

# 3) ArcFace 임베딩 생성 및 정규화
id_emb = torch.tensor(face["embedding"], dtype=torch.float16)[None].cuda()
id_emb = id_emb / torch.norm(id_emb, dim=1, keepdim=True)

# 4) Arc2Face 텍스트 인코더를 통해 ID 토큰 자리에 임베딩 주입
prompt_embeds = project_face_embs(pipeline, id_emb)


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: ./models/antelopev2/arcface.onnx recognition ['None', 3, 112, 112] 127.5 127.5


AssertionError: 

In [ ]:
# 원하는 생성 파라미터
num_images = 4
steps       = 25
guidance    = 3.0

# 1) 생성
outputs = pipeline(
    prompt_embeds=prompt_embeds,
    num_inference_steps=steps,
    guidance_scale=guidance,
    num_images_per_prompt=num_images
)

# 2) PIL.Image 리스트로 결과 획득
images = outputs.images  # length == num_images

# 3) 결과 저장 및 표시
for i, im in enumerate(images):
    fn = f"/content/output_id{i}.png"
    im.save(fn)
    display(im)      # Colab 환경에서 이미지 바로 보기
    print("saved to", fn)


In [1]:
import nbformat, glob, os

# ① 현재 작업 디렉터리(*.ipynb가 있는 곳) 자동 탐색
for path in glob.glob("*.ipynb"):
    nb = nbformat.read(path, as_version=nbformat.NO_CONVERT)
    if "widgets" in nb["metadata"]:
        nb["metadata"].pop("widgets")          # widgets 블록 삭제
        nbformat.write(nb, path)
        print(f"✅  '{path}' 수정 완료")
    else:
        print(f"ℹ️  '{path}'에는 widgets 메타데이터 없음")